LangChainMemory 
```
이전 대화 내용을 기억해서 문맥을 유지하는 역활 LangChain 0.3X부터는 LCEL 기반으로 체인을 구성, RunnableWithMessageHistory, ChatMessageHistory 등의 컴포넌트를 활용해서 세션별 대화 기록을 관리, 대화가 장기화될 경우 요약 메모리를 도입해서 과거 대화를 LLM으로 요약하고 축약된 형태로 저장해서 프롬프트의 길이문제를 해결
```

In [8]:
%pip install --quiet langchain langchain-openai python-dotenv langchain_redis

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain_core.chat_history import  InMemoryChatMessageHistory
# 메모리 객체 생성
history =  InMemoryChatMessageHistory()
history.add_user_message("안녕하세요 제 이름은 홍길동 입니다.")
history.add_ai_message('안녕하세요 홍길동님, 무엇을 도와드릴까요?')
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')

human : 안녕하세요 제 이름은 홍길동 입니다.
ai : 안녕하세요 홍길동님, 무엇을 도와드릴까요?


In [ ]:
# Radis 기반 채팅 기록 저장소
from langchain_redis import RedisChatMessageHistory
import os
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
session_id = 'user_123'
history =  RedisChatMessageHistory(session_id=session_id, redis_url=REDIS_URL)
history.add_user_message("안녕하세요 제 이름은 홍길동 입니다.")
history.add_ai_message('안녕하세요 홍길동님, 무엇을 도와드릴까요?')
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')

InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕하세요 제 이름은 홍길동 입니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요 홍길동님, 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={})])

In [18]:
# 세션기반 다중사용자 메모리 구조 구현 - 다중사용자 챗봇
# 핵심 :session_id를 키로 하는 메모리 저장소만들고 사용자의 대화는 키별로 저장한다.
from langchain_core.prompts import  ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system",'당신은 뛰어난 한국어 상담 챗봇입니다 질문에 친절하고 자세히 답변해주세요'),
    # history키로 전달된 메세지 목록은 체인 실행시 해당 위치에 넣겠다는 의미
    MessagesPlaceholder(variable_name='history'), 
    ('human','{input}')
])
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)

In [19]:
# LCEL
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

In [11]:
# 세션별 메모리 저장소를 딕셔너리로 만들고, 존재하지 않는 새로운 새션 id가 들어오면 InMemoryChatMessageHistory를 생성
# get_session_history를 구현

In [20]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# 세션 id -> 대화 기록 객체 매핑
store = {}
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    '''"세션 ID"에 해당하는 대화 기록 객체를 반환합니다.(없으면 새로 생성)'''
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 메모리를 통합한 체인 래퍼 생성
chatbot = RunnableWithMessageHistory(    
    chain,
    get_session_history,    
    input_messages_key = 'input',
    history_messages_key='history'
)

In [21]:
# 두개의 세션을 번갈아가면서 대화  RunnableWithMessageHistory 가 각 세션에 맞는 대화 기록을 관리합니다.
sessions = ['user_a','user_b']
questions = [
    '안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?',   # usre_a 첫번재 질문
    '안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?', # user_b 첫번째 질문
    '저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?', # user_a 두번째 질문
    '저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?' # user_b 두번째 질문
]
for i, question in enumerate(questions):
    session_id = sessions[i % 2]  # 세션 ID를 번갈아가며 사용
    result = chatbot.invoke({'input': question}, config={'configurable': {'session_id': session_id}})
    print(f'[{session_id}] 질문: {question}')
    print(f'[{session_id}] 챗봇: {result}\n')

[user_a] 질문: 안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?
[user_a] 챗봇: 안녕하세요, 홍길동님! 저는 여러분의 질문에 답변하고 도움을 드리기 위해 만들어진 챗봇입니다. 어떤 궁금한 점이나 도움이 필요하신 부분이 있으신가요?

[user_b] 질문: 안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?
[user_b] 챗봇: 안녕하세요, 이순신님! 저는 여러분의 질문에 답변하고, 정보 제공, 상담 등을 도와주는 챗봇입니다. 다양한 주제에 대해 이야기할 수 있으니 궁금한 점이나 도움이 필요하신 부분이 있다면 언제든지 말씀해 주세요!

[user_a] 질문: 저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?
[user_a] 챗봇: 프로그래밍을 배우고 계시다니 멋지네요! 저는 여러분의 질문에 답변하고, 정보 제공, 문제 해결, 그리고 다양한 주제에 대한 상담을 하는 역할을 하고 있습니다. 프로그래밍에 관련된 질문이나 도움이 필요하시면 언제든지 말씀해 주세요! 어떤 언어를 배우고 계신가요?

[user_b] 질문: 저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?
[user_b] 챗봇: 역사에 관심이 많으시군요! 역사적인 사건이나 인물에 대해 이야기하는 것은 정말 흥미로운 주제입니다. 저는 특정한 관심 분야가 없지만, 다양한 주제에 대한 정보를 제공할 수 있습니다. 역사, 과학, 기술, 문화 등 여러 분야에 대해 질문하시면 최대한 자세히 답변해 드리겠습니다. 어떤 역사적 사건이나 인물에 대해 이야기해 보고 싶으신가요?



In [22]:
result = chatbot.invoke({'input': "저는 철수에요, 반갑습니다."}, config={'configurable': {'session_id': 'user_c'}})
print(f'[user_c] 질문: 저는 철수에요, 반갑습니다.')
print(f'[user_c] 챗봇: {result}\n')

[user_c] 질문: 저는 철수에요, 반갑습니다.
[user_c] 챗봇: 안녕하세요, 철수님! 반갑습니다. 어떻게 도와드릴까요? 궁금한 점이나 이야기하고 싶은 것이 있다면 말씀해 주세요.



In [23]:
result = chatbot.invoke({'input': "저는 누구라구요?"}, config={'configurable': {'session_id': 'user_a'}})
result

'홍길동님이라고 말씀하셨습니다! 혹시 더 궁금한 점이나 다른 질문이 있으신가요? 도움이 필요하시면 언제든지 말씀해 주세요!'

In [24]:
result = chatbot.invoke({'input': "저는 누구라구요?"}, config={'configurable': {'session_id': 'user_b'}})
result

'이순신님이라고 말씀하셨습니다! 이순신은 조선시대의 유명한 장군으로, 임진왜란 때의 전투에서 큰 업적을 남긴 인물입니다. 혹시 이순신 장군에 대해 더 이야기하고 싶으신가요, 아니면 다른 역사적 인물이나 사건에 대해 궁금한 점이 있으신가요?'

In [25]:
result = chatbot.invoke({'input': "저는 누구라구요?"}, config={'configurable': {'session_id': 'user_c'}})
result

'철수님이라고 말씀하셨습니다! 혹시 더 궁금한 점이나 다른 이야기를 나누고 싶으신가요? 언제든지 말씀해 주세요!'

요약 메모리 구현(대화내용 자동 요약)
```
긴 대화내용을 모두 프롬프트에 기록하는 것은 비 효율적 -> 프롬프트의 길이 제한에 걸릴 가능성이 있음
Conversation Summay Memory
0.3x 버전에서는 직접 요약용 체인을 만들어서 ChatMessageHistory에 적용
```
어떻게 요약?
```
- 일정길이 이상으로 대화가 누적되면, 과거 대화를 요약해서 핵심내용만 남김
- 요약결과를 메모리에 시스템 메세지 등으로 저장->메모리 절약
- 새로운 사용자 입력시 요약된 맥락 + 최근 몇 메시지만 참고해서 llm 전달달
```

In [26]:
# 요약용 프롬프트 탬플릿
summary_prompt = ChatPromptTemplate.from_messages([
    ("system","당신은 대화 요약 전문가입니다. 대화의 주요 내용을 간결하게 요약해 주세요"),
    ("human","{conversation}") # 전체 대화내용을 하나의 문자열로 전달
])
# LCEL
summary_chain = summary_prompt | llm | StrOutputParser()

In [ ]:
# user_d 세션에 대화내용을 기록 긴 대화 생성
long_queries = [
    '안안녕, 오늘 우리 뭐하려고 했지?',
    '아 맞다 내일 회의자료 준비해야지, 회의는 몇시지?',
    '그 회의에 누가 참석하는지 기억나니?',
    '단위프로젝트 진행 상황도 공유해야 할까?',
    '최근에 이야기 했던 새로운 기능에대한 업데이트는 있어?'
]
session_id = 'user_d'
for q in long_queries:
    answer = chatbot.invoke({'input': q}, config={'configurable': {'session_id': session_id}})

print(f'요약전 user_d의 메모리 메세지 개수 : {len(store[session_id].messages)}')
print(store[session_id])  # 요약하기전 대화 내용

요약전 user_d의 메모리 메세지 개수 : 10
Human: 아녕, 오늘 우리 뭐하려고 했지?
AI: 안녕하세요! 오늘 어떤 이야기를 나누고 싶으신가요? 혹시 특정한 주제나 궁금한 점이 있으신가요? 함께 이야기해보아요!
Human: 아 맞다 내일 회의자료 준비해야지, 회의는 몇시지?
AI: 회의 시간이 기억나지 않으신가요? 회의 일정은 보통 이메일이나 캘린더에 기록되어 있을 텐데요. 확인해보시고, 혹시 다른 도움이 필요하시면 말씀해 주세요! 회의 자료 준비에 대한 조언이나 팁도 드릴 수 있습니다.
Human: 그 회의에 누가 참석하는지 기억나니?
AI: 회의 참석자에 대한 정보는 제가 알 수 없지만, 보통 회의 초대 이메일이나 캘린더에 참석자 목록이 포함되어 있을 거예요. 확인해보시면 좋을 것 같습니다. 만약 참석자 목록을 정리하는 데 도움이 필요하시거나, 회의 준비에 대한 조언이 필요하시면 말씀해 주세요!
Human: 단위프로젝트 진행 상황도 공유해야 할까?
AI: 네, 단위 프로젝트 진행 상황을 공유하는 것은 좋은 아이디어입니다. 팀원들과의 소통을 통해 프로젝트의 진행 상황을 확인하고, 필요한 지원이나 피드백을 받을 수 있습니다. 

공유할 때는 다음과 같은 내용을 포함하면 좋습니다:

1. **현재 진행 상황**: 어떤 작업이 완료되었고, 어떤 작업이 진행 중인지.
2. **문제점**: 현재 직면하고 있는 문제나 어려움이 있다면 공유하세요.
3. **다음 단계**: 앞으로의 계획이나 다음에 해야 할 작업.
4. **요청 사항**: 팀원들에게 필요한 지원이나 도움이 있다면 요청하세요.

이렇게 하면 회의에서 더 효과적으로 소통할 수 있을 것입니다. 추가로 도움이 필요하시면 언제든지 말씀해 주세요!
Human: 최근에 이야기 했던 새로운 기능에대한 업데이트는 있어?
AI: 새로운 기능에 대한 업데이트는 보통 팀 내에서 정기적으로 공유되거나 회의에서 논의됩니다. 만약 특정한 기능에 대한 업데이트가 필요하시다면, 관련 팀원이나 프로젝트 리더에게 직접 문의해 보

In [ ]:
# 전체대화 내용을 요약하고 마지막 사용자 질문-답변 쌍만 원본 유지
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
# 요약 대상 대화내용 추출(마지막 QA 쌍 제외한 이전 내용)
message = store[session_id].messages

if len(message) > 2:
    ogiginal_dialog = '\n'.join([f'{msg.type.upper()} : {msg.content}' for msg in message[ : -2] ])
else:
    ogiginal_dialog = '\n'.join([f'{msg.type.upper()} : {msg.content}' for msg in message ])

#  llm으로 요약 생성
summary_text = summary_chain.invoke({'conversation':ogiginal_dialog})
print("== 요약내용 ==")
print(summary_text)
# 기존 메모리를 요약으로 교체 : 이전내용 요약본 + 최근 QA 유지
new_history = InMemoryChatMessageHistory()
new_history.messages.append(SystemMessage(content=f'요약:{summary_text}'))
# 최근 대화의 마지막 QA쌍 복원
if len(message) >= 2:
    last_user_msg = message[-2]
    last_ai_msg = message[-1]
    if isinstance(last_user_msg, HumanMessage):
        new_history.add_user_message(last_user_msg.content)
    else:
        new_history.messages.append(last_user_msg)
    if isinstance(last_ai_msg,  AIMessage):
        new_history.add_ai_message(last_ai_msg.content)
    else:
        new_history.messages.append(last_ai_msg)
#메모리 교체
store[session_id]  = new_history

== 요약내용 ==
HUMAN은 내일 회의 자료를 준비해야 하며 회의 시간과 참석자를 확인하고 싶어한다. AI는 이메일이나 캘린더에서 이를 확인할 수 있다고 안내하고, 프로젝트 진행 상황을 공유할 때 포함할 내용으로 현재 진행 상황, 문제점, 다음 단계, 요청 사항을 제안한다.


In [34]:
len(store[session_id].messages)

3